# Challenge

Now take your Keras skills and go build another neural network. Pick your data set, but it should be one of abstract types, possibly even nonnumeric, and use Keras to make five implementations of your network. Compare them both in computational complexity as well as in accuracy and given that tradeoff decide which one you like best.

Your dataset should be sufficiently large for a neural network to perform well (samples should really be in the thousands here) and try to pick something that takes advantage of neural networks’ ability to have both feature extraction and supervised capabilities, so don’t pick something with an easy to consume list of features already generated for you (though neural networks can still be useful in those contexts).

Note that if you want to use an unprocessed image dataset, scikit-image is a useful package for converting to importable numerics.

In [11]:
import os
import numpy as np
import skimage
from skimage import io, transform
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input
from keras.models import Model
from keras.optimizers import Adam

img_size = 50
train_dir = './data/asl_train/'
test_dir =  './data/asl_test/'

def get_data(folder_path):
    imgs = []
    labels = []
    for folder_name in os.listdir(folder_path):
        if not folder_name.startswith('.'):
            if ord(folder_name[0]) >= 65 and ord(folder_name[0]) <= 90:
                label = ord(folder_name[0]) - 75
            elif folder_name == 'del':
                label = 26
            elif folder_name == 'nothing':
                label = 27
            elif folder_name == 'space':
                label = 28           
            else:
                label = 29
            for file_name in tqdm(os.listdir(folder_path + folder_name)):
                img_file = io.imread(folder_path + folder_name + '/' + file_name)
                if img_file is not None:
                    img_file = transform.resize(img_file, (img_size, img_size))
                    imgs.append(np.asarray(img_file))
                    labels.append(label)
    imgs = np.asarray(imgs)
    labels = np.asarray(labels)
    return imgs, labels

X_train, y_train = get_data(train_dir)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2) 

  0%|          | 0/3000 [00:00<?, ?it/s]/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
100%|██████████| 3000/3000 [00:10<00:00, 289.46it/s]


## MLP 

In [42]:
# 69600 images with 50*50 pixels (2500 pixels total)
# 17400 images with 50*50 pixels
new_X_train = X_train.reshape(X_train.shape[0], 2500 * 3).astype('float32')
new_X_test = X_test.reshape(X_test.shape[0], 2500 * 3).astype('float32')

# new_X_train /= 255
# new_X_test /= 255

print(new_X_train.shape[0], 'train samples')
print(new_X_test.shape[0], 'test samples')

new_y_train = keras.utils.to_categorical(y_train, 30)
new_y_test = keras.utils.to_categorical(y_test, 30)

69600 train samples
17400 test samples


In [43]:
model = Sequential()

model.add(Dense(100, activation='relu', input_shape=(2500 * 3,)))
model.add(Dropout(0.1))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 100)               750100    
_________________________________________________________________
dropout_15 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_16 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 30)                3030      
Total params: 763,230
Trainable params: 763,230
Non-trainable params: 0
_________________________________________________________________


In [44]:
history = model.fit(new_X_train, new_y_train, batch_size=150, epochs=10, verbose=1, validation_data=(new_X_test, new_y_test))
score = model.evaluate(new_X_test, new_y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 69600 samples, validate on 17400 samples
Epoch 1/10
69600/69600 [==============================] - 9s 131us/step - loss: 2.8851 - acc: 0.1360 - val_loss: 2.4430 - val_acc: 0.2425
Epoch 2/10
69600/69600 [==============================] - 6s 84us/step - loss: 2.4427 - acc: 0.2180 - val_loss: 2.1332 - val_acc: 0.3139
Epoch 3/10
69600/69600 [==============================] - 6s 82us/step - loss: 2.2907 - acc: 0.2515 - val_loss: 1.9942 - val_acc: 0.3582
Epoch 4/10
69600/69600 [==============================] - 6s 83us/step - loss: 2.1629 - acc: 0.2823 - val_loss: 1.9653 - val_acc: 0.3335
Epoch 5/10
69600/69600 [==============================] - 6s 87us/step - loss: 2.0703 - acc: 0.3061 - val_loss: 1.7550 - val_acc: 0.4160
Epoch 6/10
69600/69600 [==============================] - 6s 85us/step - loss: 2.0195 - acc: 0.3167 - val_loss: 1.7201 - val_acc: 0.4222
Epoch 7/10
69600/69600 [==============================] - 7s 96us/step - loss: 1.9461 - acc: 0.3350 - val_loss: 1.7866 - val_ac

## Convolutional Neural Network

In [ ]:
# 69600 images with 50*50 pixels (2500 pixels total)
# 17400 images with 50*50 pixels
new_X_train = X_train.reshape(X_train.shape[0], 50, 50, 3).astype('float32')
new_X_test = X_test.reshape(X_test.shape[0], 50, 50, 3).astype('float32')

# new_X_train /= 255
# new_X_test /= 255

print(new_X_train.shape[0], 'train samples')
print(new_X_test.shape[0], 'test samples')

new_y_train = keras.utils.to_categorical(y_train, 30)
new_y_test = keras.utils.to_categorical(y_test, 30)

In [38]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(50,50,3,)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(30, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 48, 48, 32)        896       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 23, 23, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 33856)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               4333696   
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
__________

In [39]:
history = model.fit(new_X_train, new_y_train, batch_size=150, epochs=10, verbose=1, validation_data=(new_X_test, new_y_test))
score = model.evaluate(new_X_test, new_y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 69600 samples, validate on 17400 samples
Epoch 1/10
69600/69600 [==============================] - 346s 5ms/step - loss: 2.4156 - acc: 0.2741 - val_loss: 1.1362 - val_acc: 0.6749
Epoch 2/10
69600/69600 [==============================] - 1060s 15ms/step - loss: 1.1419 - acc: 0.6185 - val_loss: 0.5135 - val_acc: 0.8556
Epoch 3/10
69600/69600 [==============================] - 330s 5ms/step - loss: 0.7136 - acc: 0.7549 - val_loss: 0.3010 - val_acc: 0.9190
Epoch 4/10
69600/69600 [==============================] - 326s 5ms/step - loss: 0.5236 - acc: 0.8156 - val_loss: 0.1867 - val_acc: 0.9526
Epoch 5/10
69600/69600 [==============================] - 327s 5ms/step - loss: 0.4145 - acc: 0.8537 - val_loss: 0.1274 - val_acc: 0.9669
Epoch 6/10
69600/69600 [==============================] - 328s 5ms/step - loss: 0.3331 - acc: 0.8820 - val_loss: 0.1098 - val_acc: 0.9701
Epoch 7/10
69600/69600 [==============================] - 327s 5ms/step - loss: 0.2816 - acc: 0.9006 - val_loss: 0.0621 